# Imports

In [5]:
import numpy as np
import pandas as pd
import time

# Carregando os dados

In [6]:
dtype={
       "region": object, 
       "city":object, 
       "parent_category_name":object,
       "category_name": object,
       "title":object,
       "description":object, 
       "price":np.float32, 
       "activation_date": object, 
       "user_type": object, 
       "image":object, 
       "image_top_1":np.float32, 
       "deal_probability":np.float32,
       "item_seq_number": np.int,
       "item_id": object,
       "param_1": object,
       "param_2": object,
       "param_3": object,
       "user_id": object
      }

start = time.time()
df_train = pd.read_csv('./data/train_small.csv', dtype = dtype, encoding='utf8', memory_map =True)
df_test  = pd.read_csv('./data/test_small.csv' , dtype = dtype, encoding='utf8', memory_map =True)
end = time.time()

print('Tempo de carregamento dos CSVs: '+str("%.2f" % (end - start))+'s\n')


text_columns = ['title','description']
categorical_columns = ["parent_category_name","category_name","user_type", "user_id",'param_1','param_2','param_3']

target_column = "deal_probability"

# Tratando dados faltantes
for column in text_columns:
    df_train[column].fillna(value='', inplace=True)
    df_test[column].fillna(value='', inplace=True)

# Merge dataFrames train e test
df_all_data = pd.concat([df_train[text_columns+categorical_columns],df_test[text_columns+categorical_columns]])


Tempo de carregamento dos CSVs: 0.01s



# Exibindo os dados

In [7]:
df_train.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


# Eliminando colunas desnecessárias

In [8]:
columns_to_drop =['item_id']
df_train = df_train.drop(columns=columns_to_drop)
df_test  = df_test.drop(columns=columns_to_drop)
df_train.head(5)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


# Processamento de texto corrido

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import RussianStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

#Stop words russas
stop_words_ru = open('data/stopwords.txt', encoding='utf8').read().split('\n')

def stemmed_words(doc):
    stemmer = RussianStemmer()
    analyzer = CountVectorizer().build_analyzer()
    return (stemmer.stem(w) for w in analyzer(doc))


#Pré processa os dados
for column in text_columns:
    
    start = time.time()
    
    # Criando tfIdfVectorizer
    tfidf = TfidfVectorizer(sublinear_tf=True, stop_words=stop_words_ru, analyzer=stemmed_words,
                        lowercase=True, min_df = 1, max_df=0.8)

    # fit 
    tfidf.fit(df_all_data[column])

    # transform
    df_train_column_transformed = pd.DataFrame(tfidf.transform(df_train[column]).toarray())
    df_test_column_transformed  = pd.DataFrame(tfidf.transform(df_test[column]).toarray())
     
    # apagando colunas antigas
    df_train = df_train.drop(columns=[column])
    df_test  = df_test.drop(columns=[column])
    
    # Mergeando dataFrames
    df_train = pd.concat([df_train, df_train_column_transformed], axis=1)
    df_test =  pd.concat([df_test, df_test_column_transformed], axis=1)
    
    end = time.time()
    
    print('Tempo de processamento da coluna '+column+' : '+str("%.2f" % (end - start))+'s\n')
    
df_train.head(2)

Tempo de processamento da coluna title : 0.13s

Tempo de processamento da coluna description : 0.66s



,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,item_seq_number,...,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123
0,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,400.0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,3000.0,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Processamento de colunas categóricas

In [17]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

for column in categorical_columns:
    
    start = time.time()
    
     # fit labels
    label_enc = preprocessing.LabelEncoder() 
    label_enc.fit(df_all_data[column])
    
    # transform labels
    column_label_encoded       = label_enc.transform(df_all_data[column]).reshape(1, -1)    
    train_column_label_encoded = label_enc.transform(df_train[column]).reshape(1, -1)
    test_column_label_encoded  = label_enc.transform(df_test[column]).reshape(1, -1)
     
    # fit    
    one_hot_enc = OneHotEncoder()
    one_hot_enc.fit(column_label_encoded)
    
    # transform
    df_train_column_transformed = pd.DataFrame(one_hot_enc.transform(train_column_label_encoded).toarray())
    df_test_column_transformed  = pd.DataFrame(one_hot_enc.transform(test_column_label_encoded).toarray())
    
    # apagando colunas antigas
    df_train = df_train.drop(columns=[column])
    df_test  = df_test.drop(columns=[column])
    
    # Mergeando dataFrames
    df_train = pd.concat([df_train, df_train_column_transformed], axis=1)
    df_test =  pd.concat([df_test, df_test_column_transformed], axis=1)
    
    end = time.time()
    
    print('Tempo de processamento da coluna '+column+' : '+str("%.2f" % (end - start))+'s\n')
    

ValueError: X has different shape than during fitting. Expected 81, got 49.

# Processando colunas geográficas   

In [10]:
# Coordenadas das cidades fornecidas pelo usuário FrankHerfert
# Ref.: https://www.kaggle.com/frankherfert/region-and-city-details-with-lat-lon-and-clusters/data

start = time.time()

df_coords = pd.read_csv('./data/avito_region_city_features.csv', encoding='utf8', memory_map =True)
df_coords = df_coords.drop(columns=["region_id","city_region_id","region","city_region"])
df_train  = df_train.merge(df_coords, on = "city")
df_test   = df_test.merge (df_coords, on = "city")
df_train  = df_train.drop(columns=["city","region"])
df_test   = df_test.drop(columns=["city","region"])
df_coords = None

end = time.time()

print('Tempo de merge com dataset de dados geográficos : '+str("%.2f" % (end - start))+'s\n')

df_train.head(5)

Tempo de merge com dataset de dados geográficos : 0.18s



,user_id,parent_category_name,category_name,param_1,param_2,param_3,price,item_seq_number,activation_date,user_type,...,1119,1120,1121,1122,1123,latitude,longitude,lat_lon_hdbscan_cluster_05_03,lat_lon_hdbscan_cluster_10_03,lat_lon_hdbscan_cluster_20_03
0,e00f8ff2eaf9,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,400.0,2,2017-03-28,Private,...,0.0,0.0,0.0,0.0,0.0,56.838926,60.605702,65,38,21
1,5d44315f6d90,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Верхняя одежда,> 50 (XXL),2600.0,33,2017-03-16,Private,...,0.0,0.0,0.0,0.0,0.0,56.838926,60.605702,65,38,21
2,39aeb48f0017,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,3000.0,19,2017-03-26,Private,...,0.0,0.0,0.0,0.0,0.0,53.241504,50.221246,58,36,19
3,91e2f88dd6e3,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,4000.0,9,2017-03-20,Private,...,0.0,0.0,0.0,0.0,0.0,47.235714,39.701505,46,40,9
4,3ce3c2318b16,Бытовая электроника,Товары для компьютера,Мониторы,NaN,NaN,2500.0,47,2017-03-22,Private,...,0.0,0.0,0.0,0.0,0.0,47.235714,39.701505,46,40,9


# Carregamento das features geradas pelas imagens

In [11]:
# Features calculadas em notebook externo

start = time.time()

df_images_train = pd.read_csv('./data/train_jpg.csv', encoding='utf8', memory_map =True)
df_train  = df_train.merge(df_images_train, left_on = "image", right_on = "id")
df_images_train = None
df_train  = df_train.drop(columns=["image"])
df_test["image"].fillna(value="no-image", inplace=True)
#df_test pendente

end = time.time()

print('Tempo de merge com dataset de dados das imagens : '+str("%.2f" % (end - start))+'s\n')

df_train.head(5)

Tempo de merge com dataset de dados das imagens : 26.93s



,user_id,parent_category_name,category_name,param_1,param_2,param_3,price,item_seq_number,activation_date,user_type,...,width,height,size,dullness,whiteness,average_red,average_green,average_blue,average_pixel_width,blurrness_score
0,e00f8ff2eaf9,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,400.0,2,2017-03-28,Private,...,358.0,480.0,27039.0,0.000,1.965,0.360951,0.320277,0.612833,2.329493,398.428606
1,39aeb48f0017,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,3000.0,19,2017-03-26,Private,...,360.0,480.0,30385.0,6.335,0.000,0.297394,0.366578,0.422213,3.273727,1014.477548
2,91e2f88dd6e3,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,4000.0,9,2017-03-20,Private,...,392.0,360.0,18681.0,0.000,72.020,0.703338,0.703571,0.703576,2.649518,493.921065
3,3ce3c2318b16,Бытовая электроника,Товары для компьютера,Мониторы,NaN,NaN,2500.0,47,2017-03-22,Private,...,480.0,360.0,33865.0,35.240,0.000,0.364382,0.389037,0.414090,1.414931,286.578111
4,6630d724bbf5,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,1300.0,59,2017-03-20,Private,...,360.0,480.0,60586.0,0.000,0.000,0.501076,0.527293,0.538818,2.567130,1436.013879


# Processamento de colunas temporais

In [12]:
#Convertendo datas

start = time.time()

date_column_name = 'activation_date'
df_train[date_column_name] = df_train[date_column_name].astype('datetime64[ns]')
df_test [date_column_name] = df_test[date_column_name].astype('datetime64[ns]')

# Data será transforamda em três colunas: ano - 1970, mês e dia
df_train[date_column_name] = df_train[date_column_name].map(lambda t: [t.year -1970, t.month,t.day]) 
df_train['year']  = df_train[date_column_name].map(lambda t: t[0]) 
df_train['month'] = df_train[date_column_name].map(lambda t: t[1])
df_train['day']   = df_train[date_column_name].map(lambda t: t[2])
df_train = df_train.drop(columns=[date_column_name])

df_test[date_column_name] = df_test[date_column_name].map(lambda t: [t.year -1970, t.month,t.day]) 
df_test['year']  = df_test[date_column_name].map(lambda t: t[0]) 
df_test['month'] = df_test[date_column_name].map(lambda t: t[1])
df_test['day']   = df_test[date_column_name].map(lambda t: t[2])
df_test = df_test.drop(columns=[date_column_name])

end = time.time()

print('Tempo de processamento da coluna de datas : '+str("%.2f" % (end - start))+'s\n')

df_train.head(5)

Tempo de processamento da coluna de datas : 0.14s



,user_id,parent_category_name,category_name,param_1,param_2,param_3,price,item_seq_number,user_type,image_top_1,...,dullness,whiteness,average_red,average_green,average_blue,average_pixel_width,blurrness_score,year,month,day
0,e00f8ff2eaf9,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,400.0,2,Private,1008.0,...,0.000,1.965,0.360951,0.320277,0.612833,2.329493,398.428606,47,3,28
1,39aeb48f0017,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,3000.0,19,Private,692.0,...,6.335,0.000,0.297394,0.366578,0.422213,3.273727,1014.477548,47,3,26
2,91e2f88dd6e3,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,4000.0,9,Private,3032.0,...,0.000,72.020,0.703338,0.703571,0.703576,2.649518,493.921065,47,3,20
3,3ce3c2318b16,Бытовая электроника,Товары для компьютера,Мониторы,NaN,NaN,2500.0,47,Private,2849.0,...,35.240,0.000,0.364382,0.389037,0.414090,1.414931,286.578111,47,3,22
4,6630d724bbf5,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,1300.0,59,Private,87.0,...,0.000,0.000,0.501076,0.527293,0.538818,2.567130,1436.013879,47,3,20


# Normalização dos dados

In [ ]:
'''
from sklearn.preprocessing import StandardScaler

start = time.time()

scaler = StandardScaler(copy=False)
scaler.fit(df_train)
scaler.transform(df_train)

end = time.time()

print('Tempo de processamento para normalização dos dados : '+str("%.2f" % (end - start))+'s\n')

'''

# Salvando dados processados

In [ ]:
'''
df_train.to_csv('df_train_processado.csv', encoding='utf-8', index=False)
df_test.to_csv('df_test_processado.csv', encoding='utf-8', index=False)
'''